In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import requests

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# get the HTML

url = "https://www.99acres.com/property-in-kolkata-ffid-page-1"
r = requests.get(url)
htmlContent = r.content

In [ ]:
# parse the HTML

soup = BeautifulSoup(htmlContent, 'html.parser')

In [ ]:
soup.title.text

'Property in Kolkata - Real Estate / Property for sale in Kolkata'

## Accessing only 1 page

In [ ]:
containers = soup.find_all(name='div', attrs={'class':'pageComponent srpTuple__srpTupleBox srp'})
containers[0]

<div class="pageComponent srpTuple__srpTupleBox srp" data-custominfo='{"payload":{"search_results":{"selected_entity_tuples":[{"id":"56008360","rank":1}]}}}' data-label="FSL" data-rank="1_1" id="G56008360"><div class="srpTuple__tupleDetails srpTuple__fsl"><div class="pageComponent srpTuple__thumb " data-label="PHOTO"><div class="srpTuple__photonImgSec"><picture><source alt="SGIL Residenzza Others" height="100%" srcset="https://static.99acres.com/universalapp/img/noImage.png" style="background-image:url(https://static.99acres.com/universalapp/img/noImage.png)" title="99acres" type="image/png" width="100%"/><img alt="SGIL Residenzza Others" height="100%" src="https://static.99acres.com/universalapp/img/noImage.png" style="background-image:url(https://static.99acres.com/universalapp/img/noImage.png)" title="99acres" width="100%"/></picture><div class="srpTuple__srpTupleImgCnt"><i class="iconS_Common_16 icon_squrebar"></i><span class="badges_regular_primary_white">1</span></div></div><div 

In [ ]:
first = containers[0]

In [ ]:
first.prettify()

'<div class="pageComponent srpTuple__srpTupleBox srp" data-custominfo=\'{"payload":{"search_results":{"selected_entity_tuples":[{"id":"56008360","rank":1}]}}}\' data-label="FSL" data-rank="1_1" id="G56008360">\n <div class="srpTuple__tupleDetails srpTuple__fsl">\n  <div class="pageComponent srpTuple__thumb " data-label="PHOTO">\n   <div class="srpTuple__photonImgSec">\n    <picture>\n     <source alt="SGIL Residenzza Others" height="100%" srcset="https://static.99acres.com/universalapp/img/noImage.png" style="background-image:url(https://static.99acres.com/universalapp/img/noImage.png)" title="99acres" type="image/png" width="100%"/>\n     <img alt="SGIL Residenzza Others" height="100%" src="https://static.99acres.com/universalapp/img/noImage.png" style="background-image:url(https://static.99acres.com/universalapp/img/noImage.png)" title="99acres" width="100%"/>\n    </picture>\n    <div class="srpTuple__srpTupleImgCnt">\n     <i class="iconS_Common_16 icon_squrebar">\n     </i>\n     

In [ ]:
first.find('a', {'class':'body_med srpTuple__propertyName'}).text.split('in ')[1]       # Location

'Airport'

In [ ]:
first.find('td', {'class':'list_header_bold srpTuple__spacer10'}).text                  # Property

'SGIL Residenzza'

In [ ]:
first.find('td', {'id':'srp_tuple_price'}).text.split('₹')[1].lstrip()                  # Price

'48.38  - 68.33 Lac'

In [ ]:
first.find('td', {'id':'srp_tuple_primary_area'}).text.split(' (')[0]                   # Area

'1,152-1,627 sq.ft.'

In [ ]:
first.find('td', {'id':'srp_tuple_bedroom'}).text.split(' ')[0]                         # BHK

'3'

In [ ]:
# running loop for entire 1st page containers

plocation = []
pproperty = []
pprice = []
parea = []
pbhk = []

for container in containers:
  plocation.append(container.find('a', {'class':'body_med srpTuple__propertyName'}).text.split('in ')[1])
  pproperty.append(container.find('td', {'class':'list_header_bold srpTuple__spacer10'}).text)
  pprice.append(container.find('td', {'id':'srp_tuple_price'}).text.split('₹')[1].lstrip())
  parea.append(container.find('td', {'id':'srp_tuple_primary_area'}).text.split(' (')[0])
  pbhk.append(container.find('td', {'id':'srp_tuple_bedroom'}).text.split(' ')[0] )

In [ ]:
details = list(zip(plocation, pbhk, pprice, parea, pproperty))

df = pd.DataFrame(details, columns =['Location', 'BHK', 'Price', 'Area', 'Property'])
df.head()

,Location,BHK,Price,Area,Property
0,Airport,3,48.38 - 68.33 Lac,"1,152-1,627 sq.ft.",SGIL Residenzza
1,Narendrapur,2,35.71 - 41.45 Lac,"871-1,011 sq.ft.",Su Casa Royal
2,Narkeldanga,3,1.5 - 1.5 Cr,"1,172-1,340 sq.ft.",Prasad Rare Earth
3,Narkeldanga,4,2.28 - 2.28 Cr,"1,976-2,062 sq.ft.",Prasad Rare Earth
4,Beliaghata,3,86.54 - 94.71 Lac,"1,049-1,148 sq.ft.",Bella Vista


In [ ]:
df.tail()

,Location,BHK,Price,Area,Property
20,Behala Chowrasta,3,47.59 - 59.56 Lac,"791-1,001 sq.ft.",Merlin Next
21,Tollygunge,3,97 Lac,"1,024 sq.ft.",Merlin The One
22,New Alipore,3,1.1 - 1.17 Cr,"1,105-1,168 sq.ft.",Merlin Pristine
23,Dum Dum,2,48.5 - 69.44 Lac,688-985 sq.ft.,Eden The Forest
24,Chingrighata,3,1.12 - 1.47 Cr,"859-1,116 sq.ft.",PS Jiva


## Accessing all pages

In [ ]:
pages = np.arange(1,11)   # 50 pages in total, taking in a variable

plocation = []
pproperty = []
pprice = []
parea = []
pbhk = []

In [ ]:
for page in pages:    # running for every page
  url = "https://www.99acres.com/property-in-kolkata-ffid-page-"+str(page)
  r = requests.get(url)
  htmlContent = r.content
  soup = BeautifulSoup(htmlContent, 'html.parser')
  containers = soup.find_all(name='div', attrs={'class':'pageComponent srpTuple__srpTupleBox srp'})
  sleep(randint(3,6))           # sleeping for 3 to 6 secs to control crawling

  for container in containers:
      plocation.append(container.find('a', {'class':'body_med srpTuple__propertyName'}).text.split('in ')[1])
      pproperty.append(container.find('td', {'class':'list_header_bold srpTuple__spacer10'}).text)
      try:
        pprice.append(container.find('td', {'id':'srp_tuple_price'}).text.split('₹')[1].lstrip())
      except:
        pprice.append('Price on Request')
      parea.append(container.find('td', {'id':'srp_tuple_primary_area'}).text.split(' (')[0])
      pbhk.append(container.find('td', {'id':'srp_tuple_bedroom'}).text.split(' ')[0] )

In [ ]:
details = list(zip(plocation, pbhk, pprice, parea, pproperty))

df = pd.DataFrame(details, columns =['Location', 'BHK', 'Price', 'Area', 'Property'])
df.head()

,Location,BHK,Price,Area,Property
0,Narendrapur,2,27.81 - 36.35 Lac,673-850 sq.ft.,Amaya Residences
1,Garia,2,25.08 Lac,809 sq.ft.,Richmond Enclave
2,Narkeldanga,3,1.5 - 1.5 Cr,"1,172-1,340 sq.ft.",Prasad Rare Earth
3,Narkeldanga,4,2.28 - 2.28 Cr,"1,976-2,062 sq.ft.",Prasad Rare Earth
4,Beliaghata,3,86.54 - 94.71 Lac,"1,049-1,148 sq.ft.",Bella Vista


In [ ]:
df.head(50)

,Location,BHK,Price,Area,Property
0,Narendrapur,2,27.81 - 36.35 Lac,673-850 sq.ft.,Amaya Residences
1,Garia,2,25.08 Lac,809 sq.ft.,Richmond Enclave
2,Narkeldanga,3,1.5 - 1.5 Cr,"1,172-1,340 sq.ft.",Prasad Rare Earth
3,Narkeldanga,4,2.28 - 2.28 Cr,"1,976-2,062 sq.ft.",Prasad Rare Earth
4,Beliaghata,3,86.54 - 94.71 Lac,"1,049-1,148 sq.ft.",Bella Vista
5,Tangra,3,1.3 Cr,"1,424 sq.ft.",Merlin Regalia
6,B T Road,4,85.11 Lac,"1,998 sq.ft.",Dhoot Pratham
7,Salt Lake,3,1.33 - 1.45 Cr,"1,559-1,705 sq.ft.",Merlin The Fourth
8,New Alipore,2,70 - 70.75 Lac,728-736 sq.ft.,Merlin Pristine
9,Howrah,3,21.02 - 24.33 Lac,841-973 sq.ft.,Swarna Bhoomi


In [ ]:
df.tail()

,Location,BHK,Price,Area,Property
245,Paikpara,4,100 Lac - 1.16 Cr,"1,947-2,263 sq.ft.",Meridian Splendora
246,Nayabad,3,42 Lac,"1,100 sq.ft.",Technocon Sunflower Apartment
247,Narendrapur,2,27.45 - 27.91 Lac,900-915 sq.ft.,Atri Green Valley
248,Madhyamgram,2,19.11 - 29.1 Lac,637-970 sq.ft.,Sapphire
249,Jessore Road,3,90 Lac Onwards,"1,700-1,920 sq.ft.",Emami City


In [ ]:
len(df)

250

Reviews of 250 properties have been scraped. 